# Recurrent Neural Networks

In order to learn how to work with Recurrent Neural Network we will build the ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)). Our goal is to train the conditional probability model, which will predict the next character in the sequence given the previous elements: 

$$P(c_k|\{c_1,c_2,\dots,c_{k-1}\})$$


We will work on the dataset containing the William Shakespeare playwrights. 

[![](https://upload.wikimedia.org/wikipedia/commons/a/a2/Shakespeare.jpg)](https://en.wikipedia.org/wiki/William_Shakespeare)

>Tomorrow, and tomorrow, and tomorrow,
Creeps in this petty pace from day to day,
To the last syllable of recorded time;
And all our yesterdays have lighted fools
The way to dusty death. Out, out, brief candle!
Life's but a walking shadow, a poor player
That struts and frets his hour upon the stage,
And then is heard no more. It is a tale
Told by an idiot, full of sound and fury,
Signifying nothing.

    Macbeth, Act V, scene v.

Before we start implementing code, let talk about the theory:

### Recurrent neural networks

- Comparing to the other architectures RNN networks are directed cyclic graphs.
- It means that the data could flow in the network not only in one direction - it also could be propagated through neurons in the same layer.

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

As a result RNNs are perfect tool for our task: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

The biggest problem with a Recurrent Neural Network architecture is a fact, that the information in the long sequences is often vanishing. If the necessary information is close in the chain, then the model is able to learn the relationships easily:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

But when the gap between the current neuron and the crucial information from past is large, then the network might be unable to learn properly:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

To avoid that we must redefine the model. A good solution is to use LSTM layers, which are able to control the flow of information and filter the important data:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Alternatives

Instead of LSTMs and their [modifications](https://en.wikipedia.org/wiki/Long_short-term_memory) one might use other kinds of RNN layers, e.g.  <b>Gated Recurrent Unit<b> (GRU) networks:
    
[![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Gated_Recurrent_Unit.svg)](https://en.wikipedia.org/wiki/Gated_recurrent_unit)

or more case-specific models, e.g. layers for modelling [time-series.](https://github.com/sdobber/FluxArchitectures)

### Implementation

In [4]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON

In [5]:
isfile("shakespeare.txt") ||
        download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt","shakespeare.txt")

true

In [6]:
text = collect(read("shakespeare.txt",String));
alphabet = [unique(text)..., '_'];

In [7]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [8]:
N = length(alphabet);
seqlen = 50;
batch_size = 50;

In [9]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [10]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 128),
    Dense(128, N),
    softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  #Flux.reset!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)))
  end
  return String(take!(buf))
end

#evalcb = () -> @show loss(Xs[5], Ys[5])

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#4 (generic function with 1 method)

In [11]:
sample(m, alphabet, 50)

"?gX.g.'ji F&]O'MV:d]Ilu!?&cHuRmAGBScYnN;ybBD\n;i-HX"

In [12]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:50
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model_EN.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 3
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[12]:1


loss(Xs[5], Ys[5]) = 210.26907f0


┌ Info: Epoch: 1
└ @ Main In[12]:5


_-j',agmH_DZLfCB&WF X! XmH-RJmvmMGT:!t
GQb]!q

d$qT;PMDxPq$fjJEi&fJyHo-rjsegzvGiUsdkQjbjvfiFQyb]HwF,
loss(Xs[5], Ys[5]) = 140.79349f0
wkje
fL-sDrAETArIet hih, thes ntontegithe daaon?'jho d nsTne
-ut  efesora'wAlinleuicve s nd iwp ah, 
loss(Xs[5], Ys[5]) = 133.74557f0
tDkxF lo a mee!

T rITLorounyomeff; athary ewsa mirttalledthct.

Th n moullanthsat accithehbatisga,

loss(Xs[5], Ys[5]) = 128.492f0
wZodbtow t r nt r.

'lthimy yhelowitherere, rat yine laly tou! oudd I  nin fht! widcermus; DoI Wa no


┌ Info: New best result: 127.290504
└ @ Main In[12]:10


...

┌ Info: Epoch: 19
└ @ Main In[12]:5


loss(Xs[5], Ys[5]) = 113.389824f0
dcaqls yis tlit that.
'f's the aturint, from Gthy ge sene tr-f thine, the
Dor. seing timy guing you.
loss(Xs[5], Ys[5]) = 111.965996f0
UtFHD:
Weed, thy Save my!
Go d trawt you betendewaravggronearmaly e that by torks tade,
Yamuist, inf
loss(Xs[5], Ys[5]) = 111.046684f0
wrSzCO:
Hyorest thedhaves,
ApoF ak, do wst; sbabraGusearhall ho
FANININONA:
That ver now des you.
ra


┌ Info: Epoch: 20
└ @ Main In[12]:5


loss(Xs[5], Ys[5]) = 110.12025f0
RM_. ! father shek to lleaid As, sb haibenthould,

ROLLINGil,
Wher  fachim,  ed hithose with -re woa
loss(Xs[5], Ys[5]) = 109.479065f0
3AXorinlI oo, ce,
Hace yhhs fpliit. and arying of righoughs havesepiredrep the ban's-ned.
To GRHAMAE
loss(Xs[5], Ys[5]) = 111.197495f0
Jtatdcbed we pordadith thou deatraraconjage shou bever Pesee;

KIUF IA:
Thes seart, Thatrved the seo


┌ Warning:  -> We're calling this converged.
└ @ Main In[12]:16


In [13]:
BSON.@load "char_model_EN.bson" m

In [25]:
print(sample(m, alphabet, 100))

 Lt.
Yo,  ove and faue go epauand I sam,
With but at aus.
Yorcomt air ghis strtlb ysy n mace in falt

## Extra Homework

Improve the quality of presented example <b>(5 points)</b>.